In [ ]:
import sys, os
repodir = os.path.abspath('..')
if not repodir in sys.path:
    sys.path.append(repodir)

In [ ]:
import itertools
import math

import numpy as np

from aircleaning import svg, html, produce

In [ ]:
svg.draw_scene(
    persons=5, activity=0, seed=0,
    )

In [ ]:
dim_range = range(3, 7, 1)
window_range = range(0, 6, 1)
person_range = range(1, 6, 1)
allvols = tuple(sorted(set(map(np.product, itertools.combinations_with_replacement(dim_range, 3)))))
ach_range = range(0, 16)
cadrs = tuple(sorted(set(
    math.ceil(val / 100) * 100
    for val in map(np.product, tuple(itertools.product(ach_range, allvols)))))
    )

room_combos = tuple(itertools.product(dim_range, dim_range, dim_range, window_range, person_range))
for room_combo in room_combos:
    scale = np.power(1/np.product(room_combo[:3]), 1/3) / np.power(1/np.min(allvols), 1/3)
    room = svg.draw_scene(*room_combo, size=2)
    room.projection.scale(scale)
    room.save_svg('_'.join(map(str, room_combo)), os.path.join(repodir, 'products', 'rooms'))

# outpath = os.path.join(produce.productsdir, 'costs')
# for cadr in cadrs:
#     produce.cost_analysis_by_cadr(max(100, cadr), path=outpath, name=str(cadr))

In [ ]:
page = html.Page(
    html.Script(
        '''function form_update(form){''',
        '''    var length, width, height, windows,''',
        '''        persons, room, volume, window_cleaning, mech_vent,''',
        '''        provided_cleaning, required_cleaning, activity_level,''',
        '''        extra_cleaning, extra_cleaning_cadr, acchart;''',
        '''    length = form.room_length.value;''',
        '''    width = form.room_width.value;''',
        '''    height = form.room_height.value;''',
        '''    windows = form.number_windows.value;''',
        '''    persons = form.number_people.value;''',
        '''    room = document.getElementById('room_viz');''',
        '''    room.src =''',
        '''        "https://rsbyrne.github.io/aircleaning/products/rooms/" +''',
        '''        length + "_" + width + "_" + height + "_" + windows + "_" + persons''',
        '''        + ".svg";''',
        '''    volume = form.room_length.value * form.room_width.value * form.room_height.value;''',
        '''    document.getElementById("total_volume").innerHTML = volume;''',
        '''    window_cleaning = form.number_windows.value * 200;''',
        '''    mech_vent = form.mech_vent.value * 6 * volume;''',
        '''    provided_cleaning = Math.round((window_cleaning + mech_vent) / volume);''',
        '''    document.getElementById("provided_cleaning").innerHTML = provided_cleaning;''',
        '''    activity_level = parseInt(form.activity_level.value);''',
        '''    required_cleaning = (activity_level + 1) * persons;'''
        '''    document.getElementById("required_cleaning").innerHTML = required_cleaning;'''
        '''    extra_cleaning = Math.max(0, required_cleaning - provided_cleaning);'''
        '''    document.getElementById("extra_cleaning").innerHTML = extra_cleaning;''',
        '''    extra_cleaning_cadr = Math.ceil(extra_cleaning * volume / 100) * 100;'''
        '''    document.getElementById("extra_cleaning_cadr").innerHTML = extra_cleaning_cadr;''',
        '''    acchart = document.getElementById("air_cleaner_recommendations");''',
        '''    acchart.src =''',
        '''        "https://rsbyrne.github.io/aircleaning/products/costs/" +''',
        '''        extra_cleaning_cadr''',
        '''        + ".png";''',
        '''}''',
        ),
    html.Div(
        html.Div(
            html.Form(
                html.Fieldset(
                    'How big is your room?',
                    {
                        'Length (m):': html.CapturedInput(
                            'range', 5, input_kwargs=dict(
                                min=dim_range.start, max=dim_range.stop-1, step=dim_range.step,
                                style="width: 90%;", name='room_length',
                                ),
                            ),
                        'Width (m):': html.CapturedInput(
                            'range', 5, input_kwargs=dict(
                                min=dim_range.start, max=dim_range.stop-1, step=dim_range.step,
                                style="width: 90%;", name='room_width',
                                )
                            ),
                        'Height (m):': html.CapturedInput(
                            'range', 3, input_kwargs=dict(
                                min=dim_range.start, max=dim_range.stop-1, step=dim_range.step,
                                style="width: 90%;", name='room_height',
                                ),
                            ),
                        },
                    html.Div(
                        html.Div(
                            "Total volume (m<sup>3</sup>):",
                            style="float: left; width:70%; margin-right:10px",
                            ),
                        html.Div('-', identity='total_volume', style="float: left; width:20%"),
                        style="margin-upper:10px"
                        ),
                    name='room_size',
                    style="margin: 10px;",
                    ),
                html.Fieldset(
                    "How much cleaning is already available?",
                    {
                        'Mechanical ventilation:': html.LabelledInput(
                            'Yes', html.Input('checkbox', 0, name='mech_vent'),
                            ),
                        'Number of open windows:': html.CapturedInput(
                            'range', 0, input_kwargs=dict(
                                min=window_range.start, max=window_range.stop-1, step=window_range.step,
                                style="width: 90%;", name='number_windows',
                                ),
                            ),
                        },
                    html.Div(
                        html.Div(
                            "Provided cleaning (ACH):",
                            style="float: left; width:70%; margin-right:10px",
                            ),
                        html.Div('-', identity='provided_cleaning', style="float: left; width:20%"),
                        style="margin-upper:10px"
                        ),
                    name='room_cleaning',
                    style="margin: 10px;",
                    ),
                html.Fieldset(
                    "How is the room being used?",
                    {
                        'Number of people:': html.CapturedInput(
                            'range', 1, input_kwargs=dict(
                                min=person_range.start, max=person_range.stop-1, step=person_range.step,
                                style="width: 90%;", name='number_people',
                                )
                            ),
                        'Level of activity:': html.Div(
                            html.Div(*(
                                html.LabelledInput(
                                    level,
                                    html.Input('radio', i, checked=i==0, name='activity_level'),
                                    )
                                for i, level in enumerate(('Relaxed', 'Moderate', 'Intense'))
                                ), style="display:flex;justify-content:center;align-items:center;"),
                            style="margin:10px",
                            ),
                        },
                    html.Div(
                        html.Div(
                            "Required cleaning (ACH):",
                            style="float: left; width:70%; margin-right:10px",
                            ),
                        html.Div('-', identity='required_cleaning', style="float: left; width:20%"),
                        style="margin-upper:10px"
                        ),
                    name='room_usage',
                    style="margin: 10px;",
                    ),
                html.Div(
                    "Extra cleaning required:",
                    html.Div(
                        html.Div(
                            "Air changes (ACH):",
                            style="float: left; width:70%; margin-right:10px",
                            ),
                        html.Div('-', identity='extra_cleaning', style="float: left; width:20%"),
                        style="margin-upper:10px"
                        ),
                    html.Div(
                        html.Div(
                            "CADR: (m<sup>3</sup>/hr):",
                            style="float: left; width:70%; margin-right:10px",
                            ),
                        html.Div('-', identity='extra_cleaning_cadr', style="float: left; width:20%"),
                        style="margin-upper:10px"
                        ),
                    ),
                identity='user_form',
                style="width: 45%; float: left; text-align:center;",
                ),
            html.Div(
                # svg.draw_scene(windows=2),
                html.Image(
                    "https://via.placeholder.com/150", identity="room_viz",
                    style="max-width:100%; max-height:100%;"
                    ),
                style="float: right; width: 50%; margin: 10px;",
                ),
            # style="overflow: hidden; text-align: center; margin:10px",
            style="display:flex;justify-content:center;align-items:center; margin:10px",
            )
        ),
    html.TabbedPanes(
        html.Image(
            'https://rsbyrne.github.io/aircleaning/products/0_0.png',
            identity="air_cleaner_recommendations",
            style="display:flex;justify-content:center;align-items:center; margin:10px",
            title="Air cleaner recommendations",
            ),
        html.Image(
            "https://via.placeholder.com/150",
            style="display:flex;justify-content:center;align-items:center; margin:10px",
            title="Air quality analysis",
            ),
        html.Image(
            "https://via.placeholder.com/150",
            style="display:flex;justify-content:center;align-items:center; margin:10px",
            title="Infection risk",
            ),
        ),
    html.Script('''form_update(document.getElementById("user_form"))'''),
    style="float:center;margin:auto",
    )
display(page)
page.save_html('sample')

In [ ]:
html.Image("http://localhost:8889/lab/tree/products/rooms/3_3_3_1_1.svg").save_html('test')

In [ ]:
html.Image('https://rsbyrne.github.io/aircleaning/products/0_0.png')

In [ ]:
allvols

ach in range(0, 16)